# Neural Network

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install matplotlib

In [ ]:
import utils
from utils import TrainingPlot, TimeSummary, plot_training_summary
from NNKeras import NNKeras
from sklearn.model_selection import train_test_split
import inspect
import warnings
warnings.filterwarnings('ignore')

## Step 1
### Data Preparation

#### Data Cleanup
-  Merge 64 size vectors by ignoring line breaks
-  Create subset of data by selecting non consecutive vectors

#### Cleaned data
L42023,0.04347826,0.04347826,0.,0.04347826,0.01086957,0.02173913,0.,0.02173913,0.,0.,0.,0.,0.,0.02173913,0.02173913,0.04347826,0.07608696,0.02173913,0.,0.0326087,0.01086957,0.,0.,0.0326087,0.,0.01086957,0.,0.0326087,0.,0.,0.,0.0326087,0.05434783,0.,0.01086957,0.02173913,0.04347826,0.,0.01086957,0.02173913,0.02173913,0.,0.,0.01086957,0.0326087,0.,0.04347826,0.0326087,0.01086957,0.01086957,0.,0.02173913,0.04347826,0.01086957,0.,0.01086957,0.,0.,0.,0.,0.04347826,0.02173913,0.,0.

## Network 1
### Create input dataset X and y where X has all vectors and y is one-hot vector

In [2]:
nn = NNKeras("/tf/dataset/dataset_full_rnd.csv")
X, y, unique_classes = nn.read_data()

In [3]:
print("Input Vector Shape")
print(X.shape)
print("Initialized Training Vector Shape")
print(y.shape)
print("Input Vector Head")
print(X[:3])
print("Training Vector Head")
print(y[:3])
print("Categories")
unique_classes

Input Vector Shape
(81068, 64)
Initialized Training Vector Shape
(81068, 31)
Input Vector Head
         1         2         3         4         5         6         7   \
0  0.045455  0.045455  0.000000  0.000000  0.045455  0.318182  0.000000   
1  0.055556  0.015873  0.047619  0.000000  0.000000  0.000000  0.023810   
2  0.027316  0.021378  0.015439  0.027316  0.004751  0.024941  0.008314   

         8         9         10  ...        55        56        57        58  \
0  0.000000  0.000000  0.045455  ...  0.000000  0.000000  0.045455  0.000000   
1  0.000000  0.031746  0.023810  ...  0.007937  0.000000  0.000000  0.015873   
2  0.005938  0.000000  0.013064  ...  0.011876  0.010689  0.001188  0.010689   

         59        60        61       62        63        64  
0  0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  
1  0.015873  0.023810  0.007937  0.02381  0.015873  0.023810  
2  0.004751  0.004751  0.013064  0.02019  0.016627  0.016627  

[3 rows x 64 columns]
Training

,0,indices
0,AE014075,1
1,AE006470,2
2,AE002161,3
3,AE000512,4
4,AE009442,5
5,AE002098,6
6,AE000666,7
7,BA000002,8
8,AL450380,9
9,AE000516,10


### Prepare Train and Test Data

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
print(X_train[:3])
print(y_train[:3])

             1         2        3         4         5    6     7         8   \
61310  0.078431  0.024510  0.02451  0.024510  0.000000  0.0  0.00  0.004902   
28317  0.040000  0.000000  0.02000  0.020000  0.000000  0.0  0.04  0.000000   
48590  0.032787  0.016393  0.00000  0.065574  0.016393  0.0  0.00  0.049180   

         9         10  ...    55        56   57   58   59   60        61  \
61310  0.00  0.024510  ...  0.00  0.029412  0.0  0.0  0.0  0.0  0.029412   
28317  0.04  0.040000  ...  0.04  0.020000  0.0  0.0  0.0  0.0  0.060000   
48590  0.00  0.016393  ...  0.00  0.081967  0.0  0.0  0.0  0.0  0.016393   

             62        63        64  
61310  0.019608  0.073529  0.068627  
28317  0.020000  0.020000  0.040000  
48590  0.000000  0.049180  0.000000  

[3 rows x 64 columns]
[[False False False False False False False False False False False False
  False False False False False False False False False False False  True
  False False False False False False False]
 [False Fa

## Step 2
### Network 1
-  Create neural network with single hidden layer
-  Use **Accuracy** as metric of performance  


In [5]:
inspect.getsourcelines(nn.train)
inspect.getsourcelines(nn.base_model)

(['    def base_model(self, nodes, num_output=31, kernel_regularizer=None, layer_id=None):\n',
  '        model = Sequential()\n',
  '        layer_name = None\n',
  '        for prev_node, node in zip(nodes[:-1], nodes[1:]):\n',
  '            layer_name = "{}-in-{}-n-{}".format(layer_id, str(prev_node), str(prev_node))\n',
  "            model.add(Dense(node, activation='relu', kernel_regularizer=kernel_regularizer,\n",
  '                            input_dim=prev_node, name=layer_name))\n',
  "        model.add(Dense(num_output, activation='sigmoid', name='features'))\n",
  "        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])\n",
  '        return model\n'],
 38)

In [6]:
nn.train(X, y, False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
811/811 [==============================] - 0s 298us/step
Test loss: 0.07108456995131496
Test accuracy: 0.9758561801087253


>  With just single layer we are getting accuracy of 97% Epoch=10

## Step 3
### Network 2
-  Use the same model as step 1 for neurons  = 5 to 32
-  Measure accuracy at each step to find optimal number of neurons

In [ ]:
inspect.getsourcelines(nn.train_network_2)

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split

def base_model(nodes, num_output=31, kernel_regularizer=None, layer_id=None):
    model = Sequential()
    layer_name = None
    for prev_node, node in zip(nodes[:-1], nodes[1:]):
        layer_name = "{}-in-{}-n-{}".format(layer_id, str(prev_node), str(prev_node))
        model.add(Dense(node, activation='relu', kernel_regularizer=kernel_regularizer,
        input_dim=prev_node, name=layer_name))
    model.add(Dense(num_output, activation='sigmoid', name='features'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_network_debug(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=5)
    for P in range(2, 12):
        for num_layers in range(1, 2):
            nodes = [64] + [int(P / 2)] * num_layers
            print(nodes)
            model = base_model(nodes)
            print(model.summary())
            summary = model.fit(X_train, y_train, epochs=10, verbose=0)
            score = model.evaluate(X_test, y_test)
            print('Test loss:', score[0])
            print('Test accuracy:', score[1])

train_network_debug(X, y)

In [ ]:
nn.train_network_2(X, y)

>  ### Accuraccy is already at 96% in step 1, so we add regularize function to decrease accuracy and determine optimal number of neurons as 6
>  ### P=6 

## Step 4
### Network 3
-  Find optimal number of hidden layers with number of neurons as P/2
-  Again, since the accuracy is very high, we will have to take regularizer function
-  P=6, Number of neurons = P/2

In [ ]:
inspect.getsourcelines(nn.train_network_3)

In [ ]:
nn.train_network_3(X, y)

>  ### Because of high accuracy, we can select number of layers=6

## Step 5
### Network 4
-  Without any conditions, determine optimal architecture
-  We can run the training module with P=6 and Layers=6
-  Save model for validation

In [ ]:
inspect.getsourcelines(nn.train_with_callback)

In [ ]:
nn.train_with_callback(X, y, '/tf/models/model.h5')

# Validation
### In this dataset, our neural network reaches accuracy of 96% with sample data and 98% with full dataset. Hence it is necessary to validate the accuracy of data with trained model.
### Keras allows callbacks for training visualization. Function below runs for 200 epochs with 32 neurons and saves the model

### Predict from souce data

In [ ]:
inspect.getsourcelines(nn.predict)

In [ ]:
nn.predict("/tf/dataset/ae002161.csv", unique_classes, 5, "/tf/models/model_full.h5")

In [ ]:
nn.predict("/tf/dataset/ae003852.csv", unique_classes, 5, "/tf/models/model_full.h5")

>  ### As we can see that the accuracy is as expected for the first case but for second, it is less than expected.

## Step 6
### Network 5
-  We will use same training function with each column of y as 1D matrix
-  Take average of each inidividual prediction

In [ ]:
inspect.getsourcelines(nn.single_output_score)

In [ ]:
avg_score = nn.single_output_score(X, y)
avg_score

>  ### Average accuracy of individual prediction is less than overall prediction accuracy. Hence, we can conclude that it is better to create one network with N output neurons than N networks each with one output neuron.

## Next Steps
-  ### Generic module for generating optimized network
-  ### Visualize t-SNE for hidden layers

---

## Validation with linear classification methods

In [ ]:
##############  Validation with linear classification methods

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
print(X.shape)
print(y.shape)

tmp = DecisionTreeClassifier(min_samples_leaf=10)
tmp.fit(X_train, y_train)
y_pred = tmp.predict(X_test)
print('test', accuracy_score(y_pred, y_test))
y_pred_train = tmp.predict(X_train)
print('train', accuracy_score(y_pred_train, y_train))

In [ ]:
##############  Validation with linear classification methods

from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X_train)
# print(pca.components_)
X_proj = pca.transform(X_test)

f, ax = plt.subplots(2, sharex=True)
f.set_figheight(10)
ax[0].scatter(X_proj[:, 0], X_proj[:, 1], c=np.argmax(y_test, axis=1), alpha=0.9)
ax[1].scatter(X_proj[:, 0], X_proj[:, 1], c=np.argmax(y_pred, axis=1), alpha=0.9)

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
import numpy as np


model = SVR(kernel="linear")
model.fit(X_train, np.argmax(y_train, axis=1))
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, np.argmax(y_test, axis=1))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis()
model.fit(X_train, np.argmax(y_train, axis=1))
y_pred = model.predict(X_test)
print('test acc', accuracy_score(y_pred, np.argmax(y_test, axis=1)))
y_pred_train = model.predict(X_train)
print('train acc', accuracy_score(y_pred_train, np.argmax(y_train, axis=1)))

In [ ]:
y_prob_all = model.predict_proba(X)
list(y_prob_all[y])

In [ ]:
np.sum(y_prob_all[y]<0.01) / len(y)